In [1]:
import pandas as pd
class Files_info():
    def WA_Nodes(self):
        from bs4 import BeautifulSoup as bs
        import requests as req
        
        bsr = bs(open("sh_map_v1.0.osm"),"lxml")

        nodes = bsr.find_all('node')
        print(len(nodes))

        datalist=[]
        for node in nodes:
            datalist.append([node['id'],node['lat'],node['lon']])
        df1=pd.DataFrame(datalist,columns=['origin_id','lat','lon'])
        df1['new_id']=df1.index
        df1.to_csv('Nodes.csv',index=False)

        length=len(df1)
        file=open('WA_Nodes.txt','wb')
        for i in range(length):
            print>>file,    df1.loc[i,'new_id'],df1.loc[i,'lat'],df1.loc[i,'lon']
        file.close()

        ways = bsr.find_all('way')
        way_kinds = ['motorway','motorway_link','primary','primary_link',\
                'secondary','secondary_link','tertiary','residential','living_street',\
                'service','trunk','trunk _link','unclassified']

        highways = []
        for w in ways:
            flag=False
            for tag in w.find_all('tag'):
                if tag['k']=='highway' and tag['v'] in way_kinds:
                    flag=True
                    break
            if flag:
                highways.append(w)
        def chunks(l, n):

            for i in range(0, len(l), n-1):
                yield l[i:i+n]

        final_list=[]
        for w in highways:

            isOneway = False
            category = 'unclassified'
            for tag in w.find_all('tag'):
                if tag['k']=='oneway' and tag['v'] =='yes':
                    isOneway = True
                if tag['k']=='highway':
                    category = tag['v']

            nodes=[nd['ref'] for nd in w.find_all('nd')]

            nodes_list=list(chunks(nodes,4))
            if(len(nodes_list[-1])==1):
                nodes_list.pop()    
            for nds in nodes_list:
                final_list.append([1, True,  category,nds])

        df = pd.DataFrame(final_list,columns=['id','oneway','category','nodes'])
        df['id']=df.index
        df.to_csv('highway.csv',index=False)

    


    def WA_Edges(self):
        df=pd.read_csv('Nodes.csv')
        ddf=pd.read_csv('highway.csv').loc[:,['id','nodes']]

        ddf['from']=ddf['nodes'].apply(lambda x:eval(x)[0])
        ddf['to']=ddf['nodes'].apply(lambda x:eval(x)[-1])
        ddf['cost']=ddf['to'].apply(lambda x:1)

        def get_node_lat_lon(node_id):

            node=df[df.origin_id==node_id]
            return node.iloc[0,1],node.iloc[0,2]

        def cal_distance(nodeA,nodeB):
        #     nodeA=eval(nodeA)
        #     nodeB=eval(nodeB)
            from geopy.distance import vincenty
            latA,lonA=get_node_lat_lon(nodeA)
            latB,lonB=get_node_lat_lon(nodeB)
            a=(latA,lonA)
            b=(latB,lonB)

            distance=vincenty(a, b).m
            return distance

        def distance(strlist):
            list=[eval(node_id) for node_id in eval(strlist)]

            dis=0
            for i in range(len(list)-1):
                dis=dis+cal_distance(list[i],list[i+1])
            return dis
        ddf['distance']=ddf['nodes'].apply(lambda x: distance(x))
        ddff=ddf.loc[:,['id','from','to','cost','distance','nodes']] 
        ddff.to_csv('Edges.csv',index=False)

        def get_newid(origin_id):
            return int(df[df.origin_id==int(origin_id)].loc[:,'new_id'])

        ddff['from']=ddff['from'].apply(lambda x:get_newid((x)))
        ddff['to']=ddff['to'].apply(lambda x:get_newid((x)))

        ddff.loc[:,['id','from','to','cost']].to_csv('WA_Edges.csv',index=False)

        newdf=pd.read_csv('WA_Edges.csv')
        length=len(newdf)
        file=open('WA_Edges.txt','wb')
        for i in range(length):
            print>>file,    int(newdf.loc[i,'id']),int(newdf.loc[i,'from']),int(newdf.loc[i,'to']),int(newdf.loc[i,'cost'])

        file.close()
    

    def WA_Edgegeometry(self):

        def get_node_lat_lon(node_id):
            node = f[f.origin_id==node_id]
            return str(node.iloc[0,1])+'^'+str(node.iloc[0,2])

        def format(strlist):
            list=[eval(node_id) for node_id in eval(strlist)]

            str=''
            for i in range(len(list)):
                str=str+'^'+get_node_lat_lon(list[i])
            return str

        highway_df=pd.read_csv('highway.csv')
        edge_df=pd.read_csv('Edges.csv')
        result=pd.merge(highway_df,edge_df,on=['id','nodes'])

        new=result.loc[:,['id','category','nodes']]
        new['distance']=new['category'].apply(lambda x:1)

        f=pd.read_csv('Nodes.csv')
        new['strnodes']=new['nodes'].apply(lambda x: format(x))
        new.loc[:,['id','category','distance','nodes','strnodes']].to_csv('WA_EdgeGeometry.csv',index=False)

        new=pd.read_csv('WA_EdgeGeometry.csv')
        new['distance']=new['distance'].apply(lambda x:1)
        length=len(new)
        file=open('WA_EdgeGeometry.txt','wb')
        for i in range(length):
            print>>file,    str(int(new.loc[i,'id']))+'^^'+str(new.loc[i,'category'])+'^'+str(int(new.loc[i,'distance']))+str(new.loc[i,'strnodes'])

        file.close()
    



    def deal_train(self):
        from geopy.distance import vincenty
        import os 
        import re
        output=os.listdir('input')
        row_list=[]
        for op in output:
        #     print name, group
            output_file=open('input/'+op)
            nodes=[]
            for line in output_file.readlines():
                nodes.append((line.split(',')[1],line.split(',')[2].replace('\n','')))
                          
            distance = 0
            
            for i in range(len(nodes)-1):
                node_A=nodes[i]
                node_B=nodes[i+1]
                distance=distance+vincenty(node_A, node_B).m
            row_list.append([re.match(r"input_(\d+).txt",op).group(1),distance,len(nodes)])

        df = pd.DataFrame(row_list,columns=['id','distance','nodes_length'])    
        df.to_csv('trajs_info.csv',index=False)  
    


    def map_traj_info(self):
        edf=pd.read_csv('Edges.csv')
        idf=pd.read_csv('trajs_info.csv')
        nodes=set()
        for i in range(len(edf)):
            nodes.add(edf.iat[i,1])
            nodes.add(edf.iat[i,2])


        data=[len(nodes),
              len(edf),
              edf['distance'].mean(),
              edf['distance'].sum(),
              len(idf),
              idf['nodes_length'].mean(),
              idf['distance'].mean(),
              idf['distance'].sum()]
        df = pd.DataFrame([data],columns=['number_of_road_juntions',
                                        'number_of_roads',
                                        'average_length_roads',
                                        'total_map_roads_length',
                                        'number_of_trajs',
                                        'average_points_in_one_traj',
                                       'average_length_of_one_traj',
                                        'total_length_of_trajs']) 
        df.to_csv('map_traj_info.csv',index=False) 
        print 'number_of_road_juntions',len(nodes)
        print 'number_of_roads',len(edf)
        print 'average_length_roads',edf['distance'].mean()
        print 'total_map_roads_length',edf['distance'].sum()

        print 'number_of_trajs',len(idf)
        print 'average_points_in_one_traj',idf['nodes_length'].mean()
        print 'average_length_of_one_traj',idf['distance'].mean()
        print 'total_length_of_trajs',idf['distance'].sum()
    
    def create(self):
#         self.WA_Nodes()
#         self.WA_Edges()
#         self.WA_Edgegeometry()
        self.deal_train()
        self.map_traj_info()
        
a=Files_info()
a.create()


OSError: [Errno 2] No such file or directory: 'input'